<a href="https://colab.research.google.com/github/ammobam/Display_SensorData/blob/main/(2)_%EA%B2%B0%EC%B8%A1%EC%B9%98_%EC%B2%98%EB%A6%AC%2C_%EB%B6%84%EC%82%B0_%EB%B0%8F_%EC%83%81%EA%B4%80%EA%B3%84%EC%88%98_%ED%99%95%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 전처리 소스코드
1. 분산 0인 데이터 삭제
2. 피처(독립변수) 간 종속성 높은 피처 제거

## 패키지 임포트

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
#help(pd.read_csv)

## 데이터 불러오기

In [8]:
# 데이터 불러오기
# 첫번째 열을 인덱스로 사용함
fact_data = pd.read_csv('/content/drive/MyDrive/나무플래닛/1. 데이터_디스플레이/factory_glass_2016.zip', encoding='cp949', index_col=0)

In [9]:
# 데이터 확인
fact_data.info()
fact_data.head()
# 840 컬럼 x 8145개 행이 있음
# 데이터 타입은 float과 int형으로 모두 숫자 데이터

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 840 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to R.Vac
dtypes: float64(834), int64(6)
memory usage: 52.3+ MB


,LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101,LIFT.OUT.ROL.MTR.M2.SPD.1WSI40102,LIFT.OUT.ROL.MTR.M3.SPD.1WSI40103,LEHR.DRV.MTR..M4.1.SPD.1WSI40104,LEHR.DRV.MTR..M4.2.SPD.1WSI40105,LEHR.DRV.MTR..M5.1.SPD.1WSI40106,LEHR.DRV.MTR..M5.2.SPD.1WSI40107,LEHR.DRV.MTR..M6.1.SPD.1WSI40108,LEHR.DRV.MTR..M6.2.SPD.1WSI40109,LEHR.DRV.MTR..M7.1.SPD.1WSI40110,LEHR.DRV.MTR..M7.2.SPD.1WSI40111,LEHR.DRV.MTR..M8.1.SPD.1WSI40112,LEHR.DRV.MTR..M8.2.SPD.1WSI40113,LEHR.DRV.MTR..M9.1.SPD.1WSI40114,LEHR.DRV.MTR..M9.2.SPD.1WSI40115,DRIVE.MOTOR.M10.1.SPEED.1WSI40116,LEHR.DRV.MTR..M10.2.SPD.1WSI40117,LEHR.DRV.MTR..M11.1.SPD.1WSI40118,DRIVE.MOTOR.M11.2.SPEED.1WSI40119,LEHR.DRV.MTR..M12.1.SPD.1WSI40120,LEHR.DRV.MTR..M12.2.SPD.1WSI40121,LEHR.DRV.MTR..M13.1.SPD.1WSI40122,LEHR.DRV.MTR..M13.2.SPD.1WSI40123,LEHR.DRV.MTR..M14.1.SPD.1WSI40124,LEHR.DRV.MTR..M14.2.SPD.1WSI40125,LEHR.DRV.MTR..M15.1.SPD.1WSI40126,LEHR.DRV.MTR..M15.2.SPD.1WSI40127,LEHR.DRV.MTR..M16.1.SPD.1WSI40128,DRIVE.MOTOR.M16.2.SPEED.1WSI40129,LEHR.DRV.MTR..M17.1.SPD.1WSI40130,LEHR.DRV.MTR..M17.2.SPD.1WSI40131,LEHR.DRV.MTR..M18.1.SPD.1WSI40132,LEHR.DRV.MTR..M18.2.SPD.1WSI40133,DRIVE.MOTOR.M19.1.SPEED.1WSI40134,LEHR.DRV.MTR..M19.2.SPD.1WSI40135,X1WOS45101_PV,X1WOS45102_PV,X1WOS45103_PV,X1WOS45104_PV,X1WOS45105_PV,...,S_C1_C2_L_L,S_C1_C2_L_R,S_C2_C3_U_L,S_C2_C3_U_R,S_C2_C3_L_L,S_C2_C3_L_R,S_C3_C4_U_L,S_C3_C4_U_R,S_C3_C4_L_L,S_C3_C4_L_R,S_AB1_AB11_L,S_AB1_AB11_R,S_AB11_AB7_L,S_AB11_AB7_R,S_AB7_C4_L,S_AB7_C4_R,S_C4_RET_L,S_C4_RET_R,S_RET_F_L,S_RET_F_R,STD_U_L,STD_U_R,STD_L_L,STD_L_R,STD_GLS_L,STD_GLS_R,투입.LEFT.,투입.RIGHT.,ROLLDOWN.폐기.LEFT.,진공패드.폐기.LEFT.,ROLLDOWN.폐기.RIGHT.,진공패드.폐기.RIGHT.,폐기율...,ROLLDOWN.진공패드.폐기율..LEFT.,ROLLDOWN.진공패드.폐기율..RIGHT.,ROLLDOWN.진공패드.폐기율..전체.,L.RD,L.Vac,R.RD,R.Vac
dat...date.name.,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-01 07,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.195736,0.706941,1.477203,0.410847,0.491620,0.942180,2.776941,2.492437,2.060406,2.612283,1.726404,1.461145,-1.192642,-1.134471,2.599429,2.690433,-0.365598,-0.659319,-1.963403,-1.621407,0.523950,1.951349,2.248106,2.767676,3.012706,3.069007,80,74,2,5,8,3,11.69,8.75,14.86,11.69,0.025,0.063,0.108,0.041
2016-01-01 08,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.500742,0.822657,1.495204,0.397591,0.587871,0.938186,2.888919,2.573187,2.161149,2.664726,1.682081,1.414518,-1.109125,-1.042269,2.588703,2.654307,-0.348979,-0.560103,-1.978936,-1.810022,0.566521,2.010125,2.364418,2.842630,3.022998,3.069274,92,75,0,2,10,0,7.19,2.17,13.33,7.19,0.000,0.022,0.133,0.000
2016-01-01 09,-1.772682,-1.713219,-1.734851,-1.590051,-1.636293,-1.575213,-1.636293,-1.548119,-1.824581,-1.548119,-1.8189,-1.548119,-1.779407,-1.548119,-1.779407,-1.548119,-1.806012,-1.548119,-1.775051,-1.548119,-1.776634,-1.548119,-1.766604,-1.548119,-1.775051,-1.638207,-1.728206,-1.674911,-1.641818,-1.662151,-1.636293,-1.622314,-1.653428,-1.548119,-1.807954,-1.73,-1.315444,-1.620631,-1.517655,-1.804912,...,-0.164556,0.962210,1.552127,0.460702,0.397216,0.843443,2.868644,2.486441,2.183298,2.638130,1.632955,1.359022,-0.986580,-0.928583,2.550175,2.606185,-0.456071,-0.644073,-1.878611,-1.742395,0.545936,2.002820,2.397034,2.817728,2.936967,2.969952,64,66,1,4,0,0,3.85,7.81,0.00,3.85,0.016,0.063,0.000,0.000
2016-01-01 10,-1.772682,-1.713219,-

## 데이터 전처리
- 결측치 처리
- 분산 0인 피처 제거 
- 상관관계 높은 피처 제거

### 결측치 처리

In [10]:
# 결측치 찾기
nan_data = fact_data.isnull().sum().sort_values(ascending=False)
print(nan_data)
## isnull() : 결측치를 찾아 True/False 리턴함.
    # boolean 데이터타입은 숫자로 변환하면 True -> 1, False -> 0
    # 여기서는 결측치가 있는 데이터는 1의 값을 갖게 됨
## sum() : 컬럼별로 값을 모두 더함
## sort_values() : 값을 크기별로 정렬함
    # ascending=False 옵션을 주면 내림차순으로 정렬함

ROOF.HEATING.Zone52.1JIC31152.PV        8145
UPPER.RET..Z218.TMP.1TIC42601.PV        8145
ROOF.HEATING.Zone78.1JIC31178.PV        8145
ROOF.HEATING.Zone76.1JIC31176.PV        8145
LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV    8145
                                        ... 
ROOF.HEATING.Zone45.1JIC31145.PV           0
ROOF.HEATING.Zone44.1JIC31144.PV           0
ROOF.HEATING.Zone43.1JIC31143.PV           0
ROOF.HEATING.Zone42.1JIC31142.PV           0
LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101          0
Length: 840, dtype: int64


In [11]:
# 이때 결측치 개수 컬럼은 시리즈 타입이므로 슬라이싱 됨. nan 개수 상위 20개 정도 살펴보자.
print(type(nan_data))

# 조회 결과, 7개 컬럼은 전체 8145개의 행이 모두 결측치임. 데이터가 없으므로 삭제.
print(nan_data[:20])

<class 'pandas.core.series.Series'>
ROOF.HEATING.Zone52.1JIC31152.PV         8145
UPPER.RET..Z218.TMP.1TIC42601.PV         8145
ROOF.HEATING.Zone78.1JIC31178.PV         8145
ROOF.HEATING.Zone76.1JIC31176.PV         8145
LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV     8145
DCS_Offset_RPM_M14.1WOS45114_PV          8145
DCS_Offset_RPM_M13.1WOS45113_PV          8145
DB.HEAT.TOP.4.1.2.TMP.CT.1JI40304.PV        1
DB.HEAT.TOP.5.1.2.TMP.CT.1JI40305.PV        1
DB.HEAT.TOP.7.1.2.TMP.CT.1JI40307.PV        1
DB.HEAT.BOTTOM.1.1JIC40308.PV               1
DB.HEAT.BOTTOM.2.1JIC40309.PV               1
DB.HEAT.BOTTOM.3.1JIC40310.PV               1
DB.HEAT.BOTTOM.4.1JIC40311.PV               1
ATM.FLOW.SECT.10.N2.H2.1FIC33418.PV.1       1
DB.HEAT.TOP.2.1.2.TMP.CT.1JI40302.PV        1
DB.HEAT.BOTTOM.5.1JIC40312.PV               1
DB.HEAT.BOTTOM.6.1JIC40313.PV               1
DB.HEAT.BOTTOM.7.1JIC40314.PV               1
LEHR.DRV.MTR..M18.2.SPD.1WSI40133.1         1
dtype: int64


In [12]:
# 전체 행이 결측치인 컬럼을 찾고 삭제하기

# 1. 전체 행이 결측치인 컬럼을 찾기
print(nan_data[nan_data.values == fact_data.shape[0]].index.tolist())
    # nan_data에서 다음 조건에 해당하는 컬럼을 찾음
        # nan_data.values : nan_data를 데이터 배열로 만들어줌
        # fact_data.shape -> (행 개수, 열 개수)
            # fact_data.shape[0]는 fact_data 행의 개수임
    # .index : 그때의 컬럼이름(여기선 인덱스)을 추출함
    # .tolist() : 데이터를 리스트 타입으로 변환함

# 2. 해당 컬럼삭제하기
fact_data.drop(nan_data[nan_data.values == fact_data.shape[0]].index.tolist(), axis=1, inplace=True)
    # inplace=True : drop 수행한 결과를 fact_data에 반영할지 여부 설정

# 3. 확인 - 컬럼 수가 줄어들었나 확인 (739개 -> 732개)
fact_data.info()

['ROOF.HEATING.Zone52.1JIC31152.PV', 'UPPER.RET..Z218.TMP.1TIC42601.PV', 'ROOF.HEATING.Zone78.1JIC31178.PV', 'ROOF.HEATING.Zone76.1JIC31176.PV', 'LEHR.DRV.MTR..M19.2.CUR.1WII40135.PV', 'DCS_Offset_RPM_M14.1WOS45114_PV', 'DCS_Offset_RPM_M13.1WOS45113_PV']
<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 833 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to R.Vac
dtypes: float64(827), int64(6)
memory usage: 51.8+ MB


In [13]:
# 그 외 결측치 처리
# 남은 결측치는 ffill로 채워주자
# 이 데이터는 센서 데이터의 1시간 단위 평균값이므로 전체 컬럼에 대한 평균보다는 이전 행의 값으로 결측치를 처리하는 것이 적절함
fact_data.fillna(method = 'ffill', inplace=True)
    # inplace=True : fillna 수행한 결과를 fact_data에 반영할지 여부 설정

# 확인
fact_data.isnull().sum().sort_values()

LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101         0
ROOF.HEATING.Zone50.1JIC31150.PV          0
ROOF.HEATING.Zone51.1JIC31151.PV          0
ROOF.HEATING.Zone53.1JIC31153.PV          0
ROOF.HEATING.Zone54.1JIC31154.PV          0
                                         ..
LEHR.DRV.MTR..M13.1.SPD.1WSI40122.1       0
LEHR.DRV.MTR..M13.2.SPD.1WSI40123.1       0
LEHR.DRV.MTR..M14.1.SPD.1WSI40124.1       0
DAMP.POSITION.5BAY.CTRL.1TIC33425.PV.1    0
R.Vac                                     0
Length: 833, dtype: int64

### 분산 0인 데이터 제거
- 분산이 0인 데이터를 제거하는 이유?
    - 어떤 피처의 분산이 0이라는 것은 그 피처의 데이터가 모든 행에 대해 거의 변하지 않은 것을 의미함
    - 어떤 경우에도 같은 값을 내는 컬럼이 불량률에 영향을 주고 있다고 보기 어려움
- 수행 방법
    - 방법1 : sklearn의 VarianceThreshold 사용
    - 방법2 : var() 사용
- 결과
    - 컬럼수 : 831->818


In [14]:
# 방법1
#from sklearn.feature_selection import VarianceThreshold
#selector = VarianceThreshold()
#df = selector.fit_transform(fact_data)
#print(df.shape)
#df

In [15]:
# 방법 2
# 분산 구하기
df_var = fact_data.var()    # Series
# 확인
print(df_var.describe())

# 분산이 0에 수렴하는 컬럼 확인
## 10e-10 이하면 0에 수렴하는 값으로 간주함
print(df_var[df_var.values <= 10e-10])

# 삭제
fact_data.drop(df_var[df_var.values <= 10e-10].index, axis=1, inplace=True)

# 컬럼 수 확인
print(len(fact_data.columns))

count    8.330000e+02
mean     2.144500e+00
std      2.033658e+01
min      7.538541e-27
25%      1.000219e+00
50%      1.000666e+00
75%      1.000818e+00
max      4.221950e+02
dtype: float64
LEHR.DRV.MTR..M4.2.CUR.1WII40105.PV     7.538541e-27
LEHR.DRV.MTR..M9.1.CUR.1WII40114.PV     7.538541e-27
LEHR.DRV.MTR..M9.2.CUR.1WII40115.PV     7.538541e-27
LEHR.DRV.MTR..M10.1.CUR.1WII40116.PV    7.538541e-27
LEHR.DRV.MTR..M14.1.CUR.1WII40124.PV    7.538541e-27
LEHR.DRV.MTR..M17.2.CUR.1WII40131.PV    7.538541e-27
TMP.TIN..BAY.4.LEFT.1TI30203.PV         7.538541e-27
dtype: float64
826


### 피처 간 종속성 높은 피처 제거
- 종속성 높은 피처를 제거하기 위해 레이블을 제외한 피처 간 상관계수를 확인하고 0.9 이상인 피처는 1개만 남기고 제거하고자 함
- 상관관계 : 피처 간의 종속된 정도
- 상관관계가 높은 피처를 제거하는 이유?
    - 두 피처 간의 상관관계가 높다는 것은, 하나의 피처 값이 다른 피처의 값에 큰 영향을 주고있음을 의미함
    - 두 피처는 동일한 원인에 기인하여 변하는 것으로 추측할 수 있음
    - 이를 제거하지 않고 두면 사실상 같은 의미인 데이터가 모델링에 여러 번 반영됨
    - 사실상 종속관계에 있는 피처들이 모델링에 크게 기여하는 것과 같음
    - 모델링에 영향을 미치는 원인들이 모두 비슷한 중요도로 반영되게 하려면 종속성이 낮은 피처들만을 이용하여 모델을 만드는 것이 타당함
- 여기서는 피처 간 상관계수의 절대값이 0.9 이상인 경우를 종속된 것으로 봄


In [16]:
# 모든 컬럼에 대해 상관계수를 구하고 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거

# 모든 컬럼에 대해 상관계수의 절대값 구하기
corr_arr = abs(fact_data.corr())

# 상관계수가 0.9 이상인 컬럼 중 1개만 남기고 모두 제거
del_list = []
## i : 상관계수 데이터 프레임의 컬럼
for i, col in enumerate(corr_arr.columns):
    ## j : 상관계수 데이터 프레임의 인덱스
    for j in range(len(corr_arr.index)):
        ## i행 피처와 j열 피처가 동일한 피처인 경우, 자기 자신에 대한 상관계수는 당연히 1이므로 삭제하지 않음
        if i == j:
            continue
        ## i행 피처의 상관계수가 이미 삭제 리스트에 있는 경우 넘어감
        elif corr_arr.index[i] in del_list:           
            break
        ## 만약 j행 피처와 i열 피처의 상관계수가 0.9 이상인 경우, j행을 삭제 리스트에 추가함
        ## 코드가 위에서 아래로 순차적으로 실행되기 때문에
        ## i행 피처에 대한 상관계수가 0.9 이상인 경우 위에서 break 처리를 받음
        ## 즉, (i, j) 상관계수가 0.9인 경우 j컬럼만 삭제리스트에 추가되고 i컬럼은 삭제리스트에 추가되지 않음
        # print(i, j, corr_arr.index[j], np.abs(corr_arr.iloc[j,i]))
        if np.abs(corr_arr.iloc[j,i]) >= 0.9:
            del_list.append(corr_arr.index[j])
            # print(corr_arr.index[j], '-->', corr_arr.index[i])
# 확인
len(del_list)

679

In [17]:
# del_list에 있는 컬럼은 제외
total_col = fact_data.columns.tolist()
col_remain = []
for c in total_col:
    if c not in del_list:
        col_remain.append(c)
len(col_remain)

382

In [18]:
# 저장
fact_data = fact_data[col_remain]
fact_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8145 entries, 2016-01-01 07 to 2016-12-31 23
Columns: 382 entries, LIFT.OUT.ROL.MTR.M1.SPD.1WSI40101 to ROLLDOWN.진공패드.폐기율..RIGHT.
dtypes: float64(376), int64(6)
memory usage: 23.8+ MB
